In [1]:
import os
os.chdir(os.path.join(os.environ["RNB_PLANNING_DIR"], "src"))

## init combined robot config

In [2]:
from pkg.controller.combined_robot import *
from pkg.project_config import *

crob = CombinedRobot(robots_on_scene=[
    RobotConfig(0, RobotType.indy7, None,
                INDY_IP)]
              , connection_list=[False])

connection_list
[False]


## create scene builder

In [3]:
from pkg.geometry.builder.scene_builder import SceneBuilder
s_builder = SceneBuilder(None, base_link="base_link")
# s_builder.reset_reference_coord(ref_name="floor")

## get ghnd with detected robot config

In [4]:
# xyz_rpy_robots = s_builder.detect_items(level_mask=[DetectionLevel.ROBOT])
xyz_rpy_robots = {"indy0": ((0,0,0), (0,0,-np.pi/2))}
crob.update_robot_pos_dict(xyz_rpy_robots=xyz_rpy_robots)
gscene = s_builder.create_gscene(crob)

Unknown tag "hardwareInterface" in /robot[@name='custom_robots']/transmission[@name='indy0_tran0']/actuator[@name='indy0_motor0']
Unknown tag "hardwareInterface" in /robot[@name='custom_robots']/transmission[@name='indy0_tran1']/actuator[@name='indy0_motor1']
Unknown tag "hardwareInterface" in /robot[@name='custom_robots']/transmission[@name='indy0_tran2']/actuator[@name='indy0_motor2']
Unknown tag "hardwareInterface" in /robot[@name='custom_robots']/transmission[@name='indy0_tran3']/actuator[@name='indy0_motor3']
Unknown tag "hardwareInterface" in /robot[@name='custom_robots']/transmission[@name='indy0_tran4']/actuator[@name='indy0_motor4']
Unknown tag "hardwareInterface" in /robot[@name='custom_robots']/transmission[@name='indy0_tran5']/actuator[@name='indy0_motor5']


Please create a subscriber to the marker
publication OK
published: [0, 0, 0, 0, 0, 0]


## add environment

In [5]:
from pkg.geometry.geometry import *

In [6]:
floor = gscene.create_safe(GEOTYPE.BOX, "floor", "base_link", (3,3,0.01), (0,0,0), 
                           rpy=(0,0,0), color=(0.8,0.8,0.8,0.5), display=True, fixed=True, collision=True)
wall = gscene.create_safe(GEOTYPE.BOX, "wall", "base_link", (3,3,0.01), (0,0.2,0), 
                           rpy=(np.pi/2,0,0), color=(0.8,0.8,0.8,0.5), display=True, fixed=True, collision=True)
wp1 = gscene.create_safe(GEOTYPE.BOX, "wp1", "base_link", (0.1,0.1,0.01), (0.2,-0.5,0.),rpy=(0,0,np.pi/2), 
                         color=(0.8,0.2,0.2,1), display=True, fixed=True, collision=False)
wp2 = gscene.create_safe(GEOTYPE.BOX, "wp2", "base_link", (0.1,0.1,0.01), (-0.2,-0.5,0), rpy=(0,0,np.pi/2), 
                         color=(0.8,0.2,0.2,1), display=True, fixed=True, collision=False)
track = gscene.create_safe(GEOTYPE.BOX, "track", "base_link", (0.1,0.5,0.01), (0,-0.5,0), rpy=(0,0,np.pi/2), 
                           color=(1,0.7,0.7,0.5), display=True, fixed=True, collision=False)
goal = gscene.create_safe(GEOTYPE.BOX, "goal", "base_link", (0.1,0.1,0.01), (-0.4,-0.3,0), 
                          rpy=(0,0,0), color=(0.8,0.8,0.2,1), display=True, fixed=True, collision=False)

Please create a subscriber to the marker


In [7]:
gtems = s_builder.add_robot_geometries(color=(0,1,0,0.5), display=True, collision=True, exclude_link=["panda1_link7"])

## add brush

In [8]:
brush_body = gscene.create_safe(gtype=GEOTYPE.BOX, name="brush_body", link_name="base_link", 
                                dims=(0.2,0.07,0.02), center=(0.0,-0.5,0.045), rpy=(0,0,np.pi/2), 
                                color=(0.7,0.7,0.3,1), display=True, collision=True, fixed=False)
brush_handle = gscene.create_safe(gtype=GEOTYPE.BOX, name="brush_handle", link_name="base_link", 
                                  dims=(0.2,0.03,0.05), center=(0,0,0.035), rpy=(0,0,0), 
                                  color=(0.7,0.7,0.3,1), display=True, collision=True, fixed=False, parent="brush_body")
brush_face = gscene.create_safe(gtype=GEOTYPE.BOX, name="brush_face", link_name="base_link", 
                                dims=(0.19,0.06,0.03), center=(0,0,-0.025), rpy=(0,0,0), 
                                color=(0.8,0.8,0.8,1), display=True, collision=False, fixed=False, parent="brush_body")

## init planning scene

In [9]:
from pkg.planning.scene import PlanningScene
pscene = PlanningScene(gscene, combined_robot=crob)

## Register binders

In [10]:
from pkg.planning.constraint.constraint_actor import Gripper2Tool, PlacePlane, SweepTool

In [11]:
gscene.create_safe(gtype=GEOTYPE.SPHERE, name="grip0", link_name="indy0_tcp", 
                 dims=(0.01,)*3, center=(0,0,0.14), rpy=(-np.pi/2,0,0), color=(1,0,0,1), display=True, collision=False, fixed=True)

In [12]:
pscene.create_binder(bname="grip0", gname="grip0", rname="indy0", _type=Gripper2Tool, point=(0,0,0), rpy=(0,0,0))
pscene.create_binder(bname="floor", gname="floor", _type=PlacePlane)
pscene.create_binder(bname="goal_bd", gname="goal", _type=PlacePlane, point=(0,0,0.005), rpy=(0,0,0))
pscene.create_binder(bname="brush_face", gname="brush_face", _type=SweepTool, point=(0,0,-0.015), rpy=(0,0,0))

## add objects

In [13]:
from pkg.planning.constraint.constraint_subject import CustomObject, Grasp2Point, PlacePoint, SweepPoint, SweepTask

In [14]:
brush_handle = gscene.NAME_DICT["brush_handle"]
brush_face = gscene.NAME_DICT["brush_face"]
brush = pscene.create_object(oname="brush", gname="brush_body", _type=CustomObject, 
                             action_points_dict = {"handle": Grasp2Point("handle", brush_handle, [0,0,0], [np.pi/2,0,0]),
                                                   "face": PlacePoint("face", brush_face, [0,0,-0.015], [0,0,0])})

In [15]:
sweep = pscene.create_object(oname="sweep", gname="floor", _type=SweepTask, 
                             action_points_dict = {"wp1": SweepPoint("wp1", wp1, [0,0,0.005], [0,0,0]),
                                                   "wp2": SweepPoint("wp2", wp2, [0,0,0.005], [0,0,0])})

## Grasp checker

In [16]:
from pkg.planning.filtering.grasp_filter import GraspChecker
gcheck = GraspChecker(pscene, end_link_couple_dict= {"indy0_tcp": ["indy0_tcp", "indy0_link6"], "base_link":["base_link"]})

## Reach SVM

In [17]:
from pkg.planning.filtering.reach_filter import ReachChecker
rcheck = ReachChecker(pscene)

### planning pipeline

In [18]:

from pkg.planning.pipeline import PlanningPipeline
ppline = PlanningPipeline(pscene)

from pkg.planning.motion.moveit.moveit_planner import MoveitPlanner
mplan = MoveitPlanner(pscene, [gcheck, rcheck])
mplan.update_gscene()
ppline.set_motion(mplan)

# from pkg.planning.task.a_star import TaskAstar
# tplan = TaskAstar(pscene)
# tplan.prepare()
# ppline.set_sampler(tplan)
from pkg.planning.task.rrt import TaskRRT
tplan = TaskRRT(pscene)
tplan.prepare()
ppline.set_sampler(tplan)

## ui

In [19]:
from pkg.ui.ui_broker import *

# start UI
ui_broker = UIBroker.instance()
ui_broker.initialize(ppline, s_builder)
ui_broker.start_server()

ui_broker.set_tables()

Dash is running on http://0.0.0.0:8050/

 * Serving Flask app "pkg.ui.dash_launcher" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


## Test plan

In [20]:
from pkg.planning.constraint.constraint_common \
            import sample_redundancy, combine_redundancy

initial_state = pscene.update_state(crob.home_pose)

In [21]:
mplan.reset_log(True)
with GlobalTimer.instance().block("full"):
    # goal_nodes = [("goal",)]
    goal_nodes = [("goal", 2)]
    ppline.search(initial_state, goal_nodes, verbose=True, display=False, dt_vis=0.01, timeout_loop=300, multiprocess=False, timeout=1)
    schedules = ppline.find_schedules()
    schedules_sorted = ppline.sort_schedule(schedules)
    snode_schedule = ppline.idxSchedule2SnodeScedule(schedules_sorted[0])

node: ('floor', 0)->('floor', 1) = fail
node: ('floor', 0)->('floor', 1) = fail
node: ('floor', 0)->('grip0', 0) = fail
node: ('floor', 0)->('grip0', 0) = success
branching: 0->1 (0.04/300.0 s, steps/err: 45(41.2650108337 ms)/0.00135446774831)
node: ('grip0', 0)->('floor', 0) = fail
node: ('floor', 0)->('floor', 1) = fail
node: ('grip0', 0)->('grip0', 1) = success
branching: 1->2 (0.08/300.0 s, steps/err: 53(37.1789932251 ms)/0.00174260903279)
node: ('grip0', 0)->('grip0', 1) = success
branching: 1->3 (0.12/300.0 s, steps/err: 52(39.6280288696 ms)/0.00171866767324)
node: ('grip0', 0)->('floor', 0) = fail
node: ('grip0', 0)->('floor', 0) = fail
node: ('grip0', 0)->('floor', 0) = fail
node: ('grip0', 0)->('goal', 0) = success
branching: 1->4 (0.17/300.0 s, steps/err: 49(45.8869934082 ms)/0.00112653168777)
node: ('floor', 0)->('floor', 1) = fail
node: ('grip0', 0)->('grip0', 1) = success
branching: 1->5 (0.22/300.0 s, steps/err: 51(42.5381660461 ms)/0.0011301478594)
node: ('grip0', 0)->('

In [22]:
for k,v in mplan.result_log.items():
    print("{}: {}% ({})".format(k, np.mean(v)*100, len(v)))

GraspChecker: 87.1794871795% (39)
planning: 92.0% (25)
ReachChecker: 73.5294117647% (34)


In [23]:
print(GlobalTimer.instance())

full: 	3528.0 ms/1 = 3527.646 ms (3527.646/3527.646)
test_connection: 	3505.0 ms/49 = 71.53 ms (0.117/1002.314)
GraspChecker: 	16.0 ms/39 = 0.413 ms (0.365/0.568)
ReachChecker: 	9.0 ms/34 = 0.251 ms (0.207/0.379)
planning: 	3465.0 ms/25 = 138.606 ms (14.407/1001.448)



In [37]:
ppline.play_schedule(snode_schedule, period=0.01)

## Text execution

In [21]:
# crob.reset_connection([True])

connection_list
[True]
Connect: Server IP (192.168.0.63)


## unsynced execution

In [23]:
ppline.execute_schedule(snode_schedule, vel_scale=0.5)

binder: floor
rname: None
binder: None
grasp_dict
{'indy0': False}
grasp_seq
[('indy0', False)]
Connect: Server IP (192.168.0.63)
binder: floor
rname: None
binder: None
grasp_dict
{'indy0': False}
grasp_seq
[('indy0', False)]
Connect: Server IP (192.168.0.63)
go
Connect: Server IP (192.168.0.63)
binder: grip0
rname: indy0
binder: None
grasp_dict
{'indy0': True}
grasp_seq
[('indy0', True)]
Connect: Server IP (192.168.0.63)
go
Connect: Server IP (192.168.0.63)
binder: grip0
rname: indy0
binder: brush_face
rname: None
grasp_dict
{'indy0': True}
grasp_seq
[('indy0', True)]
Connect: Server IP (192.168.0.63)
go
Connect: Server IP (192.168.0.63)
binder: grip0
rname: indy0
binder: brush_face
rname: None
grasp_dict
{'indy0': True}
grasp_seq
[('indy0', True)]
Connect: Server IP (192.168.0.63)
go
Connect: Server IP (192.168.0.63)
binder: goal_bd
rname: None
binder: brush_face
rname: None
grasp_dict
{'indy0': False}
grasp_seq
[('indy0', False)]
Connect: Server IP (192.168.0.63)
go
Connect: Server 

## synced execution

In [ ]:
## 
# with RvizPublisher(gscene, []) as rpub:
#     ppline.execute_schedule(snode_schedule, rviz_pub=rpub)

## Disconnect stereo

In [ ]:
stereo.disconnnect()